## 1. TextBlob：快速自然语言处理的“被低估的魔法师”

In [4]:
from textblob import TextBlob

text = TextBlob("I absolutely love how Python makes automation effortless!")
print(f'Sentiment: {text.sentiment}')
print(f'Noun Phrases: {text.noun_phrases}')
print(f'Corrected:: {TextBlob('Python is a powrful programing language!').correct()}')

Sentiment: Sentiment(polarity=0.625, subjectivity=0.6)
Noun Phrases: ['python', 'automation effortless']
Corrected:: Python is a powerful programming language!


## 2. spaCy：为讨厌慢代码的开发者准备的工业级 NLP

In [ ]:
import spacy
from spacy.matcher import Matcher
from spacy.language import Language

# 加载中文模型（需要提前安装）
# python -m spacy download zh_core_web_sm
nlp = spacy.load("zh_core_web_sm")

# 添加默认句子分割器（基于 "。！？"）
if "sentencizer" not in nlp.pipe_names:
    nlp.add_pipe("sentencizer", first=True)

# 进一步增强中文分句能力（处理 ，；等情况）
@Language.component('custom_seg')
def custom_seg(doc):
    for i, token in enumerate(doc[:-1]):
        # 逗号但情感转折词附近 → 分句
        if token.text in ["，", ",", ";", "；"]:
            doc[i+1].is_sent_start = True

        # 强制分句点
        if token.text in ["？", "！", "。", "…"]:
            doc[i+1].is_sent_start = True

    return doc

if 'custom_seg' not in nlp.pipe_names:
    nlp.add_pipe("custom_seg", after="sentencizer")

# 示例文本
text = "保利集团在广州开发的保利天空之城位置很好，位于市中心，且距离地铁 3 号线很近，周边学校资源不错，就是物业服务差。"

doc = nlp(text)
matcher = Matcher(nlp.vocab)

# ===========
# 1. 命名实体识别（提取地点、机构、楼盘名等）
# ===========
entities = [(ent.text, ent.label_) for ent in doc.ents]

# ===============================================
# 2. 地铁线路提取
# ===============================================
matcher = Matcher(nlp.vocab)
pattern_metro = [{"TEXT": "地铁"}, {"IS_DIGIT": True}, {"TEXT": "号线"}]
matcher.add("METRO_LINE", [pattern_metro])
matches = matcher(doc)
metros = [doc[start:end].text for _, start, end in matches]

# ===============================================
# 3. 分类（交通、教育、物业、区位等）
# ===============================================
categories = {
    "交通": ["地铁", "公交", "出行"],
    "教育": ["学校", "小学", "中学", "教育"],
    "物业": ["物业", "服务", "管理"],
    "区位": ["市中心", "位置", "地段", "周边"],
}

detected_categories = []
for cat, kws in categories.items():
    if any(k in text for k in kws):
        detected_categories.append(cat)

# ===============================================
# 4. 情感分析（强化版 scoring + 句子级）
# ===============================================
positive_words = ["很好", "不错", "很近", "便利", "舒服", "满意"]
negative_words = ["差", "不好", "一般", "糟糕", "吵", "很远"]

sentiment_score = 0
sentence_scores = []

for sent in doc.sents:
    s_score = 0
    sent_text = sent.text

    for w in positive_words:
        if w in sent_text:
            s_score += 1

    for w in negative_words:
        if w in sent_text:
            s_score -= 1

    sentence_scores.append((sent_text, s_score))
    sentiment_score += s_score

# 最终情感
if sentiment_score > 0:
    sentiment = "positive"
elif sentiment_score < 0:
    sentiment = "negative"
else:
    sentiment = "neutral"

# ===============================================
# 最终输出
# ===============================================
print("🔎 楼盘相关实体:", entities)
print("🚇 地铁线路:", metros)
print("🏷 分类:", detected_categories)
print("💬 情感得分:", sentiment_score)
print("📝 各句子得分:")
for idx, (sentence, score) in enumerate(sentence_scores):
    print(f'\t#{idx+1}-{sentence}: {score}')
print("📘 情感:", sentiment)

🔎 楼盘相关实体: [('广州', 'GPE'), ('3', 'CARDINAL')]
🚇 地铁线路: ['地铁 3 号线']
🏷 分类: ['交通', '教育', '物业', '区位']
💬 情感得分: 2
📝 各句子得分:
	#1-保利集团在广州开发的保利天空之城位置很好，: 1
	#2-位于市中心，: 0
	#3-且距离地铁 3 号线很近，: 1
	#4-周边学校资源不错，: 1
	#5-就是物业服务差。: -1
📘 情感: positive


## 3. LlamaIndex（GPT Index）：把文档变成可查询的记忆系统

In [ ]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
import os

os.environ['OPENAI_API_KEY'] = 'your APIkey'

# 1. 加载文档（支持 docx / pdf / txt / md）
documents = SimpleDirectoryReader(
    input_dir="company_policies"
).load_data()

# 2. 构建向量索引（Embedding + Node parsing）
index = VectorStoreIndex.from_documents(documents)

# 3. 创建查询引擎
query_engine = index.as_query_engine()

# 4. 提问示例：自动检索相关制度并总结回答
response = query_engine.query(
    "公司的加班计算规则是什么？有哪些特别说明？"
)

print("AI 回答：")
print(response)


TypeError: Can't instantiate abstract class ArkLLM without an implementation for abstract methods 'achat', 'acomplete', 'astream_chat', 'astream_complete', 'chat', 'metadata', 'stream_chat', 'stream_complete'

## 4. Polars + NLP：你不知道却最强的文本处理组合

In [ ]:
import polars as pl

df = pl.read_csv('comments.csv')

keywords = ['python', 'api', 'programming', 'script', 'automation']
filtered = df.filter(
    pl.col('comment').str.contains('|'.join(keywords), literal=False)
)

print(filtered)

## 5. KeyBERT：基于 Transformer 的关键词提取

In [1]:
from keybert import KeyBERT
import os

os.environ["HF_HUB_DISABLE_SYMLINKS_WARNING"] = "1"

kw_model = KeyBERT()
keywords = kw_model.extract_keywords(
    'Python makes automation incredibly simple.',
    top_n=3
)

print(keywords)

[('python', 0.7005), ('automation', 0.6758), ('simple', 0.2128)]


## 6. Transformers：无所不能的 NLP 工具箱

In [ ]:
from transformers import pipeline

classifier = pipeline('zero-shot-classification')
result = classifier(
    "This bug caused our production server to crash.",
    candidate_labels=['finance', 'software issue', 'support', 'feature request']
)

print(result)

No model was supplied, defaulted to facebook/bart-large-mnli and revision d7645e1 (https://huggingface.co/facebook/bart-large-mnli).
Using a pipeline without specifying a model name and revision in production is not recommended.


model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

d:\ProgramData\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\admin\.cache\huggingface\hub\models--facebook--bart-large-mnli. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

Device set to use cpu


{'sequence': 'This bug caused our production server to crash.', 'labels': ['software issue', 'support', 'feature request', 'finance'], 'scores': [0.7345870733261108, 0.14557555317878723, 0.07310790568590164, 0.04672941565513611]}
